In [ ]:
from __future__ import print_function
import gym
import gym_kheperaposition
import numpy as np
import math
import sys
import matplotlib.pyplot as plt

Vmax=0.05
Wmax=math.pi/4

Kr_Prop=0.025   
K1=0.1
Kp=0.75
Ki=0.00007 
PI=3.1415 

def random_agent(steps=800):
    episode_reward = 0
    ErrorAcumulado=0
    env = gym.make('KheperaPositionControl-v0')
    env.MaxSteps = steps
    d, Oc = env.reset()
    #env.render()
    
    data = {}
    paso = []
    infoOc = []
    infoD = []
    Lineal = []
    Angular = []
    xc = []
    yc = []
    
    for e in range(steps):
        error = math.atan2(math.sin(Oc), math.cos(Oc))
        p = ((math.pi-math.fabs(error))/math.pi)
        v = min(K1*d*p, Vmax)
        if p > 0.9 and d > Kr_Prop:
            v = Vmax
        if d < Kr_Prop:
            v = 0
            w = 0
        ErrorAcumulado = error + ErrorAcumulado
        w = Kp*math.sin(Oc)+Ki*ErrorAcumulado
        Vl = (v-(w*0.1)/2)*48
        Vr = (v+(w*0.1)/2)*48
        action = [Vl, Vr]
        state, reward, done, info = env.step(action)
        d, Oc = state
        #env.render()
        #print(reward)
        print(reward, end='\r')
        sys.stdout.flush()
        
        episode_reward += reward
        x = info['xc']
        y = info['yc']
        
        paso.append(e)
        infoOc.append(float(Oc))
        infoD.append(float(d))
        Lineal.append(float(v))
        Angular.append(float(w))
        xc.append(float(x))
        yc.append(float(y))  
        
        if done:
            print('Reward of the episode is: ',episode_reward)
            break
    env.close()
   
    data={'paso' : paso,
          'Oc' : infoOc,
          'Distance' : infoD,
          'Lineal' : Lineal,
          'Angular' : Angular,
          'xc' : xc,
          'yc' : yc}
    
    #print(data)
    return data

data2 = random_agent()
d, t = [np.array(data2['Distance']), 0.05*np.array(data2['paso'])]

IAE = np.trapz(abs(d),t)
ISE = np.trapz(d**2,t)
ITAE = np.trapz(t*abs(d),t)
ITSE = np.trapz(t*(d**2),t)

IPC = [ISE, IAE, ITSE, ITAE]

print(IAE)
print(ISE)
print(ITAE)
print(ITSE)